# Worked Example
The following notebook is a worked example, where we run the code from begining to end. <br>
<br>
We assume that you have read the github readme and have created and activated the conda hotspots environment.<br>
<br>
We suggest that you follow along with the example in a terminal window. Copy and paste the commands from this notebook into the terminal window. The example was produced on a linux machine so your slash(/) might be (\\). Remember this is more of a recipie than a python notebook.<br>

## Move to the scripts directory

All of the commands used in this example are in the hotspots/scripts directory. Each person has a specific location where the hotspots code is for us this is:


`cd python/hotspots/scripts`

## Create a location for your data

You'll need to create a location to store the data and the computations in: 

`mkdir ../exampledata/KFWS20190324/` <br>
`mkdir ../exampledata/KFWS20190324/raw` <br>
`mkdir ../exampledata/KFWS20190324/hotspots_raw` <br>
`mkdir ../exampledata/KFWS20190324/MCIT` <br>
`mkdir ../exampledata/KFWS20190324/StormID` <br>
`mkdir ../exampledata/KFWS20190324/hotspots_assigned` <br>

## Download some data

We have provided a script to help you access level 2 data from the USA NWS archive. The script can only download the data from a single day. If you want data crosses from 23Z to 00Z you'll need to run the script twice. We do this in our example. Also the hours specificed on the command line are really strings not integers so 0Z is 00 and 2Z is 02. Example below:

`python getRadarData.py -o ../exampledata/KFWS20190324/raw -d 20190324 -s 22 -e 24 -r KFWS`

`python getRadarData.py -o ../exampledata/KFWS20190324/raw -d 20190325 -s 00 -e 01 -r KFWS`

## Create the raw hotspots

The next task is to create the raw hotspots, but to do that you first need to determine the height of the -10C CAPPI. There are many ways to do this. You can use model data or a nearby sounding. We use the MDM data in the raw directory and convert the model sounding from the center point into above mean sea level in meters. The code to do this is only in C++. (sorry) You can get a nearby sounding from many places including https://weather.cod.edu/analysis/

For the KFWS case on 20190324 we have selected the AMSL height of 5040 as the height of the -10C. <br> This piece of python is quite a bit slower than the rest, because it has the most work to do. 

`python compute_zdr_hotspots.py -i ../exampledata/KFWS20190324/raw -o ../exampledata/KFWS20190324/hotspots_raw/ -r KFWS -c 5040`

## Create the raw MCIT data

Next we create some Storm Cell Identification data. There are many storm cell identification methods. The one we use is MCIT (see github readme file) and is a watershed method. 

`python compute_MCIT.py -i ../exampledata/KFWS20190324/raw -o ../exampledata/KFWS20190324/MCIT/ -r KFWS`

## Create Tracked Storm Cell Identification Data

We have implemented a simple overlap tracking algorithm to link storm cells identified at time (t) to storm cells at time (t-1). There are many methods to do this, but this one is particularly easy to understand. 

`python compute_StormID.py -i ../exampledata/KFWS20190324/MCIT -o ../exampledata/KFWS20190324/StormID/`

## Assign Hotspots to StormID
We use the storm cell data to identify which hotspots are updrafts by assigning hotspots to the storms. We assign them to tracked storm cells so that the hotspots themselves can be tracked over time. Storm Cells are larger than hotspots and likely easier to track, but the storm cell identification and tracking process can also have errors in it that break hotspot time order. You can also try and assign hotspots at time (t) to hotspots at time (t-1) but we did not find that to be robust. 

`python compute_assigned_hotspots.py -c ../exampledata/KFWS20190324/StormID -o ../exampledata/KFWS20190324/hotspots_assigned/ -d ../exampledata/KFWS20190324/hotspots_raw/
`

## Create Plots of our work
We have created scripts that make both individual *.png files of the data as well as animated gif movies. 

`mkdir ../exampledata/KFWS20190324/plots` <br>
`mkdir ../exampledata/KFWS20190324/plots/hotspots` <br>
`mkdir ../exampledata/KFWS20190324/plots/stormid` <br>


`python plot_hotspots_netcdf.py -g -o ../exampledata/KFWS20190324/plots/hotspots/ -d ../exampledata/KFWS20190324/hotspots_assigned/`


`python plot_stormid_netcdf.py -g -o ../exampledata/KFWS20190324/plots/stormid/ -d ../exampledata/KFWS20190324/StormID/`

## View your plots and enjoy